In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets, transforms, models
import numpy as np
from torch.autograd import grad

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [23]:
# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your jackfruit dataset (assuming a directory structure for each class)
train_dataset = datasets.ImageFolder(root=r'C:\Users\GAUTAM RAJPUROHIT\AIES MINI PROJECT FINAL\MINIPROJECT_AIES\Jackfruit_Diseasese_Prevention\train', transform=transform)
test_dataset = datasets.ImageFolder(root=r'C:\Users\GAUTAM RAJPUROHIT\AIES MINI PROJECT FINAL\MINIPROJECT_AIES\Jackfruit_Diseasese_Prevention\test' , transform=transform)

# Split into train and validation sets
train_loader = data.DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=20, shuffle=False)



In [25]:
class DenseNetMAML(nn.Module):
    def __init__(self, num_classes=3):
        super(DenseNetMAML, self).__init__()
        self.densenet = models.densenet121(pretrained=True)  # Load pretrained DenseNet
        self.densenet.classifier = nn.Linear(self.densenet.classifier.in_features, num_classes)  # Adjust the classifier

    def forward(self, x):
        return self.densenet(x)

# Initialize the network
model = DenseNetMAML().to(device)


In [27]:
def maml_update(model, loss, inner_lr):
    # Perform inner loop updates (fine-tuning on support set)
    grads = grad(loss, model.parameters(), create_graph=True)
    updated_weights = []
    for param, grad_param in zip(model.parameters(), grads):
        updated_weights.append(param - inner_lr * grad_param)
    return updated_weights

def apply_weights(model, updated_weights):
    # Apply updated weights to the model
    for param, updated_param in zip(model.parameters(), updated_weights):
        param.data = updated_param.data

def meta_train(model, train_loader, outer_lr=0.001, inner_lr=0.01, num_steps=5):
    optimizer = optim.Adam(model.parameters(), lr=outer_lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(20):
        model.train()
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            # Split support and query sets (5-shot learning)
            support_images = images[:5]
            support_labels = labels[:5]
            query_images = images[5:]
            query_labels = labels[5:]
            
            # Forward pass on support set
            support_outputs = model(support_images)
            support_loss = criterion(support_outputs, support_labels)
            
            # Perform inner loop update (fine-tuning)
            updated_weights = maml_update(model, support_loss, inner_lr)

            # Forward pass on query set using updated weights
            apply_weights(model, updated_weights)  # Apply updated weights to model
            query_outputs = model(query_images)
            query_loss = criterion(query_outputs, query_labels)

            # Perform outer loop update (meta-learning)
            optimizer.zero_grad()
            query_loss.backward()
            optimizer.step()
        
        print(f'Epoch [{epoch+1}/{20}], Loss: {query_loss.item():.4f}')


In [29]:
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on test images: {accuracy:.2f}%')

# After training, evaluate the model



In [ ]:
# Train the model using MAML
meta_train(model, train_loader)

Epoch [1/20], Loss: nan
Epoch [2/20], Loss: nan
Epoch [3/20], Loss: nan
Epoch [4/20], Loss: nan
Epoch [5/20], Loss: nan
Epoch [6/20], Loss: nan


In [107]:
evaluate(model, test_loader)

Accuracy of the model on test images: 96.33%


In [17]:
# Save the trained model
torch.save(model.state_dict(), 'jackfruit_disease_model.pth')


In [19]:
from PIL import Image
class DenseNetMAML(nn.Module):
    def __init__(self, num_classes=3):
        super(DenseNetMAML, self).__init__()
        self.densenet = models.densenet121(pretrained=False)
        self.densenet.classifier = nn.Linear(self.densenet.classifier.in_features, num_classes)

    def forward(self, x):
        return self.densenet(x)

# Load the trained model
model = DenseNetMAML()
model.load_state_dict(torch.load('jackfruit_disease_model.pth', map_location=torch.device('cpu')))
model.eval()  # Set the model to evaluation mode

# Define class names (same as during training)
class_names = ['Healthy', 'Algae Leaf Spot Disease', 'Black Spot Disease']

# Function to preprocess an image
def transform_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Function to make a prediction
def predict(image_path):
    image_tensor = transform_image(image_path)
    output = model(image_tensor)
    _, predicted = torch.max(output, 1)
    return class_names[predicted.item()]


C:\Users\GAUTAM RAJPUROHIT\AIES MINI PROJECT FINAL\MINIPROJECT_AIES\Jackfruit_Diseasese_Prevention\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\GAUTAM RAJPUROHIT\AppData\Local\Temp\ipykernel_12900\2842983106.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode un